## Pré processamento dos dados

### Bibliotecas e base de dados

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)              
pd.set_option('display.max_columns', None)          
pd.set_option('display.max_colwidth', 50)           
pd.set_option('display.width', 1000)                 
pd.set_option('display.float_format', '{:.2f}'.format)  
pd.set_option('display.colheader_justify', 'left')  
pd.set_option('display.float_format', '{:,.2f}'.format)
def estilo_minimalista(df):
    return df.style.set_properties(**{
        'font-size': '8px',    
        'border-color': 'black', 
        'border-width': '0.4px',   
        'border-style': 'solid',
        'padding': '1px',        
    })
sns.set_theme(style="white", palette="deep", context="talk",font_scale=0.8)
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
object_columns = ['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao',
                  'Setores Censitários', 'Status', 'DH_Descricao', 'DM_Descricao',
                  'DA_Descricao', 'DA_Polui/cont da água', 'DA_Polui/cont do ar',
                  'DA_Polui/cont do solo', 'DA_Dimi/exauri hídrico',
                  "DA_Incêndi parques/APA's/APP's", 'PEPL_Descricao', 'PEPR_Descricao',
                  'Categoria', 'Grupo', 'Subgrupo', 'Tipo', 'Subtipo']

dtype = {col: 'object' for col in object_columns}

df_eventos = pd.read_csv(
    "https://raw.githubusercontent.com/brunagmoura/PrevisorReconhecimento/refs/heads/main/df_eventos_desastres_rec_nrec.csv",
    sep=';',
    decimal=',',
    dtype=dtype)

df_eventos.head(5)

,Protocolo_S2iD,Nome_Municipio,Sigla_UF,regiao,Data_Registro,Data_Evento,codigo_ibge,Setores Censitários,Status,DH_Descricao,DH_MORTOS,DH_FERIDOS,DH_ENFERMOS,DH_DESABRIGADOS,DH_DESALOJADOS,DH_DESAPARECIDOS,DH_OUTROS AFETADOS,DH_total_danos_humanos,DM_Descricao,DM_Uni Habita Danificadas,DM_Uni Habita Destruidas,DM_Uni Habita Valor,DM_Inst Saúde Danificadas,DM_Inst Saúde Destruidas,DM_Inst Saúde Valor,DM_Inst Ensino Danificadas,DM_Inst Ensino Destruidas,DM_Inst Ensino Valor,DM_Inst Serviços Danificadas,DM_Inst Serviços Destruidas,DM_Inst Serviços Valor,DM_Inst Comuni Danificadas,DM_Inst Comuni Destruidas,DM_Inst Comuni Valor,DM_Obras de Infra Danificadas,DM_Obras de Infra Destruidas,DM_Obras de Infra Valor,DM_total_danos_materiais,DA_Descricao,DA_Polui/cont da água,DA_Polui/cont do ar,DA_Polui/cont do solo,DA_Dimi/exauri hídrico,DA_Incêndi parques/APA's/APP's,PEPL_Descricao,PEPL_Assis_méd e emergên(R$),PEPL_Abast de água pot(R$),PEPL_sist de esgotos sanit(R$),PEPL_Sis limp e rec lixo (R$),PEPL_Sis cont pragas (R$),PEPL_distrib energia (R$),PEPL_Telecomunicações (R$),PEPL_Tran loc/reg/l_curso (R$),PEPL_Distrib combustíveis(R$),PEPL_Segurança pública (R$),PEPL_Ensino (R$),PEPL_total_publico,PEPR_Descricao,PEPR_Agricultura (R$),PEPR_Pecuária (R$),PEPR_Indústria (R$),PEPR_Comércio (R$),PEPR_Serviços (R$),PEPR_total_privado,PE_PLePR,Empenhado,DensidadePop,Hab,Area,Ano,PIB,Categoria,Grupo,Subgrupo,Tipo,Subtipo,COBRADE
0,SC-F-4200903-12200-20141226,Angelina,SC,Sul,2015-01-02 00:00:00,2015-01-02,4200903,NaN,Reconhecido,Moradores da comunidade que ficarem desprovido...,0,0,0,0,0,0,1500,1500,Os danos são representados pelas avarias/destr...,0,0,0.00,0.00,0,0.00,0.00,0,0.00,0,0,0.00,0,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"No caso exposto, os prejuízos públicos são rep...",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"Quebra/perda da produção agrícola/pecuária, oc...",0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,10.72,"5,358.00",500.00,"2,015.00","103,375.00",Natural,Hidrológico,Enxurradas,NaN,NaN,12200
1,MS-F-5000708-12200-20141222,Anastácio,MS,Centro-oeste,2015-01-05 00:00:00,2015-01-05,5000708,NaN,Reconhecido,Ficaram prejudicados um total de 800 famílias ...,0,0,0,0,0,0,3200,3200,Foram danificadas tum total de 8 pontes que in...,0,0,0.00,0.00,0,0.00,0.00,0,0.00,0,0,0.00,0,0,0.00,0.00,0.00,"6,271,293.52","6,271,293.52",NaN,NaN,NaN,NaN,NaN,NaN,Foram danificadas tum total de 8 pontes que in...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,332,356.82",0.00,0.00,0.00,"2,332,356.82","Produtores da Agricultura Familiar, produtores...","326,156.77","1,630,783.87",0.00,0.00,0.00,"1,956,940.64","4,289,297.46",NaN,8.28,"24,114.00","2,913.18","2,015.00","394,893.00",Natural,Hidrológico,Enxurradas,NaN,NaN,12200
2,RS-F-4301925-12200-20150101,Barra do Rio Azul,RS,Sul,2015-01-06 00:00:00,2015-01-06,4301925,NaN,Não reconhecido,NaN,0,0,0,0,0,0,0,0,Danos e destruição de 35 Bueiros e danos em 15...,0,0,0.00,0.00,0,0.00,0.00,0,0.00,0,0,0.00,0,0,"54,435.57",30.00,5.00,"1,041,889.69","1,096,325.26",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,11.54,"1,696.00",147.00,"2,015.00","42,201.00",Natural,Hidrológico,Enxurradas,NaN,NaN,12200
3,RS-F-4311601-12200-20141231,Liberato Salzano,RS,Sul,2015-01-07 00:00:00,2015-01-07,4311601,NaN,Reconhecido,NaN,0,0,0,0,0,0,0,0,Devido a alta precipitação pluviométrica e con...,0,0,0.00,0.00,0,0.00,0.00,0,0.00,0,0,0.00,0,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"Destruição das vias de acesso do Município, co...",0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,740,983.83",0.00,0.00,0.00,"1,740,983.83",Devido as fortes precipitações ocorridas ocor...,"9,718,266.89",0.00,0.00,0.00,0.00,"9,718,266.89","11,459,250.72",NaN,19.46,"4,781.00",245.63,"2,015.00","116,822.00",Natural,Hidrológico,Enxurradas,NaN,NaN,12200
4,BA-F-2931608-12200-20141216,Teolândia,BA,Nordeste,2015-01-07 00:00:00,2015-01-07,2931608,NaN,Reconhecido,05 pessoas deram 

### Exclusão de variáveis

In [4]:
# Excluir variáveis que não serão adicionadas ao modelo

df_eventos = df_eventos.drop(
    ['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao', 'Data_Registro', 'Data_Evento', 'codigo_ibge',
     'Setores Censitários', 'COBRADE', "Ano"], axis=1)

In [5]:
#Excluir colunas que tenham mais do que 5% de dados ausentes
colunas_mantidas = df_na_counts[df_na_counts['% de dados ausentes'] < 5]['Variável']
df_eventos = df_eventos[colunas_mantidas]
print(df_eventos.columns)

NameError: name 'df_na_counts' is not defined

### Encoding das variáveis categóricas

In [34]:
encoder = OneHotEncoder(sparse_output=False, drop=None)  
categorias = ['Subgrupo', 'Categoria', 'Grupo']
one_hot_encoded = encoder.fit_transform(df_eventos[categorias])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorias))

df_eventos = pd.concat([df_eventos.drop(columns=categorias), one_hot_df], axis=1)

df_eventos.head(3)

,PEPL_Distrib combustíveis(R$),PEPL_total_publico,PEPL_Ensino (R$),PEPL_Segurança pública (R$),DH_DESAPARECIDOS,PEPL_Tran loc/reg/l_curso (R$),PEPR_Agricultura (R$),PEPL_distrib energia (R$),PEPL_Sis cont pragas (R$),PEPL_Telecomunicações (R$),PEPR_Indústria (R$),PEPR_Pecuária (R$),PEPL_sist de esgotos sanit(R$),PEPR_Comércio (R$),PEPR_Serviços (R$),PEPR_total_privado,PE_PLePR,DH_MORTOS,DensidadePop,Hab,Area,PEPL_Sis limp e rec lixo (R$),PEPL_Assis_méd e emergên(R$),PEPL_Abast de água pot(R$),DM_Inst Serviços Destruidas,DH_total_danos_humanos,DM_Uni Habita Danificadas,DM_Uni Habita Destruidas,DM_Uni Habita Valor,DM_Inst Saúde Danificadas,DM_Inst Saúde Destruidas,DM_Inst Saúde Valor,DM_Inst Ensino Danificadas,DM_Inst Ensino Destruidas,DM_Inst Ensino Valor,DM_Inst Serviços Danificadas,DM_Inst Serviços Valor,DH_OUTROS AFETADOS,DM_Inst Comuni Danificadas,DM_Inst Comuni Destruidas,DM_Inst Comuni Valor,DM_Obras de Infra Danificadas,DM_Obras de Infra Destruidas,DM_Obras de Infra Valor,DM_total_danos_materiais,DH_DESALOJADOS,DH_DESABRIGADOS,DH_ENFERMOS,DH_FERIDOS,Status,Subgrupo_Alagamentos,Subgrupo_Colapso de edificações,Subgrupo_Desastres relacionados à contaminação da água,Subgrupo_Enxurradas,Subgrupo_Epidemias,Subgrupo_Erosão,Subgrupo_Incêndios urbanos,Subgrupo_Infestações/Pragas,Subgrupo_Inundações,Subgrupo_Movimento de massa,Subgrupo_Rompimento/colapso de barragens,Subgrupo_Seca,Subgrupo_Sistemas de Grande Escala/Escala Regional,Subgrupo_Temperaturas Extremas,Subgrupo_Tempestades,Subgrupo_Terremoto,Subgrupo_Transporte aquaviário,Subgrupo_Transporte rodoviário,Subgrupo_nan,Categoria_Natural,Categoria_Tecnológico,Grupo_Biológico,Grupo_Climatológico,Grupo_Desastres Relacionados a Incêndios Urbanos,Grupo_Desastres Relacionados a Produtos Perigosos,Grupo_Desastres relacionados a obras civis,Grupo_Desastres relacionados a transporte de passageiros e cargas não perigosas,Grupo_Geológico,Grupo_Hidrológico,Grupo_Meteorológico
0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,10.72,"5,358.00",500.00,0.00,0.00,0.00,0,1500,0,0,0.00,0.00,0,0.00,0.00,0,0.00,0,0.00,1500,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,Reconhecido,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
1,0.00,"2,332,356.82",0.00,0.00,0,"2,332,356.82","326,156.77",0.00,0.00,0.00,0.00,"1,630,783.87",0.00,0.00,0.00,"1,956,940.64","4,289,297.46",0,8.28,"24,114.00","2,913.18",0.00,0.00,0.00,0,3200,0,0,0.00,0.00,0,0.00,0.00,0,0.00,0,0.00,3200,0,0,0.00,0.00,0.00,"6,271,293.52","6,271,293.52",0,0,0,0,Reconhecido,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
2,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,11.54,"1,696.00",147.00,0.00,0.00,0.00,0,0,0,0,0.00,0.00,0,0.00,0.00,0,0.00,0,0.00,0,0,0,"54,435.57",30.00,5.00,"1,041,889.69","1,096,325.26",0,0,0,0,Não reconhecido,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
